In [ ]:
#pip install scikit-learn
#pip install lightgbm

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv("/Users/farhat/Documents/Project/ProcessedData/fullData.csv")
data.head(10)

,Patient_Id,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,sig_30,sig_31,sig_32,sig_33,sig_34,sig_35,sig_36,sig_37,sig_38,SepsisLabel
0,p016161,74.0,100.0,37.11,NaN,101.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,p016161,72.0,98.0,37.11,NaN,81.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,p016161,75.0,100.0,37.11,NaN,83.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,p016161,63.0,100.0,37.11,NaN,64.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,p016161,61.0,100.0,37.17,NaN,66.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,p016161,68.0,100.0,37.17,NaN,63.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,p016161,64.0,100.0,37.17,NaN,94.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,p016161,66.0,100.0,37.17,NaN,83.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,p019252,80.0,100.0,34.45,109.5,89.0,72.0,12.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,p019252,80.0,100.0,35.70,90.0,68.0,54.0,10.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [40]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X.head()

,Patient_Id,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,sig_29,sig_30,sig_31,sig_32,sig_33,sig_34,sig_35,sig_36,sig_37,sig_38
0,p016161,74.0,100.0,37.11,NaN,101.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p016161,72.0,98.0,37.11,NaN,81.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p016161,75.0,100.0,37.11,NaN,83.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p016161,63.0,100.0,37.11,NaN,64.0,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p016161,61.0,100.0,37.17,NaN,66.0,NaN,14.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Assuming you have X (features) and y (target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(1264344, 82)
(316086, 82)
(1264344,)
(316086,)


In [22]:
y_train

1306747    0
1293817    0
1279612    0
760830     0
530872     0
          ..
259178     0
1414414    0
131932     0
671155     0
121958     0
Name: SepsisLabel, Length: 1264344, dtype: int64

In [23]:
print(y_train.sort_index())

0          1
1          1
3          1
4          1
5          1
          ..
1580423    0
1580424    0
1580425    0
1580426    0
1580428    0
Name: SepsisLabel, Length: 1264344, dtype: int64


In [24]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

estimator = lgb.LGBMClassifier(
    objective='binary',
    metric='binary_logloss',
    boosting_type='gbdt'
)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.15],
    'max_depth': [4, 6, 8, 12, 16],
    'num_leaves': [15, 31, 63]
}

grid = GridSearchCV(estimator, param_grid, cv=5, scoring='accuracy', verbose=1)
grid.fit(X_train, y_train)

results = pd.DataFrame(grid.cv_results_)
results['cv'] = grid.cv
results.to_csv('LightGB_Results.csv')

print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)


Fitting 5 folds for each of 45 candidates, totalling 225 fits
Best params: {'learning_rate': 0.15, 'max_depth': 12, 'num_leaves': 63}
Best score: 0.9865954199759687


In [26]:
y_pred = grid.best_estimator_.predict(X_test)
y_pred = (y_pred >= 0.5).astype(int)

y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [27]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9872882696481338
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    309265
           1       0.84      0.51      0.63      6821

    accuracy                           0.99    316086
   macro avg       0.92      0.75      0.81    316086
weighted avg       0.99      0.99      0.99    316086



In [28]:
from sklearn.metrics import confusion_matrix

In [30]:
y_test.value_counts()

SepsisLabel
0    309265
1      6821
Name: count, dtype: int64

In [33]:
cm = confusion_matrix(y_test,y_pred=y_pred)
cm

array([[308614,    651],
       [  3367,   3454]])

In [37]:
lightgbm_testing_best_param = pd.DataFrame({
    'model' : 'lightgbm',
    'num_leaves': grid.best_estimator_.num_leaves,
    'max_depth': grid.best_estimator_.max_depth,
    'learning_rate': grid.best_estimator_.learning_rate,
    'accuracy': accuracy_score(y_test, y_pred),
    'tn':cm[0,0],
    'fp':cm[0,1],
    'fn':cm[1,0],
    'tp':cm[1,1]
}, index=[0])

In [38]:
lightgbm_testing_best_param

,model,num_leaves,max_depth,learning_rate,accuracy,tn,fp,fn,tp
0,lightgbm,63,12,0.15,0.987288,308614,651,3367,3454


In [25]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([154566,   3477]))

In [21]:
print (y_test)
print (y_pred)

70024     0
181934    0
724581    0
199837    0
581915    0
         ..
665469    0
452072    0
493677    0
536585    0
405136    0
Name: SepsisLabel, Length: 158043, dtype: int64
[0 0 0 ... 0 0 0]


In [22]:
pd.DataFrame({
    'Sepsis Label' : y_test,
    'Sepsis Predicted':y_pred
})

,Sepsis Label,Sepsis Predicted
70024,0,0
181934,0,0
724581,0,0
199837,0,0
581915,0,0
...,...,...
665469,0,0
452072,0,0
493677,0,0
536585,0,0


In [ ]:
dtrain = lgb.Dataset(X_train, label=y_train)

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1
}

# 5-fold CV
cv_results = lgb.cv(params, dtrain, num_boost_round=100, nfold=5, stratified=True, shuffle=True, seed=42)

# View results
print("Best log loss:", cv_results.keys())


Best log loss: dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv'])
